In [1]:
import os
import sys
import pandas
from matplotlib import pyplot
import numpy
import scipy.special
from seaborn import boxplot, stripplot, violinplot
from dask.distributed import Client, progress
import dask.dataframe 
import dask.delayed
import dask.bag

In [2]:
LRSC = os.path.expanduser('~/proj/long-rna-seq-condor')
SCQC = os.path.expanduser('~/proj/single-cell-qc')

for p in [LRSC, SCQC]:
    if p not in sys.path:
        sys.path.append(p)

In [3]:
from woldrnaseq.models import load_experiments
from singleqc.tube_likelihood import (
    compute_log_likelihoods,
    read_concentrations,
    make_spike_success_table,
    optimize_by_run,
    log_likelihood,
    chi,
    prob,
)

In [4]:
ls all_analysis_vdir/exp*.tsv

all_analysis_vdir/experiment_vdir.tsv


In [5]:
def load_exclude_list():
    by_run = load_experiments([
        os.path.expanduser('~/proj/C1_mouse_limb_combined/all_analysis_vdir/experiment_vdir.tsv')
    ])

    to_exclude = set()
    for name, row in by_run.iterrows():
        for exclude_name in ['run1', 'run2', 'run9']:
            if exclude_name in name:
                to_exclude.update(row.replicates)

    return to_exclude
to_exclude = load_exclude_list()

In [6]:
def compute_likelihood():
    data = make_combined_quantification(filtered, 'single', 'FPKM', concentrations)
    data = log_likelihood(data)
    data = data.sort_values(by="run_LR", ascending=True)
    data = chi(data)

    data.to_csv(target_name, sep='\t', index=False)    

In [7]:
q = pandas.read_csv(
    #'paper_analysis_vdir/macrophage_gene_FPKM.csv',
    'paper_analysis_vdir/high psmc_gene_FPKM.csv',
    usecols=lambda x: x != 'gene_name',
    dtype={'gene_id': str},
    skiprows=[1,2,3],
).set_index('gene_id').loc["gSpikein_ERCC-00002":'gSpikein_ERCC-00171']
#q = q.drop('gene_name', axis=1)
print(q.shape)
q.head()


(96, 32)


,19908_C1,19909_D8,19911_F3,20032_G3,20026_A9,20048_E12,20046_C10,18275_F1,18274_E1,20038_F3,...,18260_C7,18270_A11,20037_E8,20048_E6,18273_D12,20035_C9,18272_C10,19915_B7,18271_B6,18274_E11
gene_id,,,,,,,,,,,,,,,,,,,,,
gSpikein_ERCC-00002,4425.48,15568.31,6621.19,10169.98,8360.52,6634.30,15253.47,19037.35,24519.72,7830.35,...,7257.17,22307.90,12774.56,14411.34,22661.94,9140.67,30683.18,9315.92,22758.54,38628.72
gSpikein_ERCC-00003,1587.99,2421.92,2060.94,2215.97,2001.68,1779.88,4755.83,987.95,1786.57,1654.49,...,1369.32,1120.74,1953.42,2571.72,1733.34,1578.36,2128.91,1956.76,1380.39,3128.00
gSpikein_ERCC-00004,22428.68,23536.13,31445.22,18084.09,21851.66,17868.33,43529.54,10425.92,14581.49,24084.70,...,8630.72,13631.93,31442.90,48473.62,16288.20,25859.91,24351.33,28920.06,12154.46,26778.76
gSpikein_ERCC-00007,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
gSpikein_ERCC-00009,1252.73,1758.88,1073.03,3220.14,1997.34,2378.11,2696.91,1920.47,1996.42,845.60,...,3282.87,1814.20,895.26,1710.06,2129.19,1113.16,2689.59,911.93,2232.92,3619.57


In [8]:
q[sorted(set(q.columns) - to_exclude)].shape

(96, 14)

In [9]:
def make_spike_success_table(library_data, concentrations, quantification_name, run_name, tube_type):
    spikes = concentrations.merge(library_data, how='inner', left_index=True, right_index=True)
    success = spikes[quantification_name] > 0
    spikes = pandas.DataFrame.assign(
        spikes,
        run=run_name,
        tube_type=tube_type,
        success=success,
    )
    return spikes


In [10]:
def make_combined_quantification(quantification, tube_type, quantification_name, concentrations, sep=','):
    """Read a combined quantification files gene_id vs library_id

    this is a gene_id vs library_id tables, if there is a column named "gene_name" it
    will be ignored.
    """
    data = []
    for column in quantification.columns:
        spikes = make_spike_success_table(
            quantification[column].to_frame(quantification_name),
            concentrations,
            quantification_name,
            column,
            tube_type)
        data.append(spikes)
        
    return pandas.concat(data)


In [19]:
concentrations = dask.dataframe.from_pandas(read_concentrations(os.path.expanduser('~/proj/single-cell-qc/singleqc/ENCSR535LMC.tsv')), chunksize=5)

df = make_combined_quantification(q, 'single', 'FPKM', concentrations.compute())
print(df.shape)
df.head()

(2944, 6)


,length,concentration,FPKM,run,tube_type,success
gene_id,,,,,,
gSpikein_ERCC-00002,1061,2031.750000,4425.48,19908_C1,single,True
gSpikein_ERCC-00003,1023,126.984375,1587.99,19908_C1,single,True
gSpikein_ERCC-00004,523,1015.875000,22428.68,19908_C1,single,True
gSpikein_ERCC-00009,984,126.984375,1252.73,19908_C1,single,True
gSpikein_ERCC-00012,994,0.015501,0.00,19908_C1,single,False


In [20]:
df.to_csv('paper_analysis_vdir/high_psmc_gene_FPKM_success.csv')

In [12]:
class ComputeLogLikelihood:    
    def __init__(self):
        self.K = numpy.arange(0, 61)
        self.K_factorial = scipy.special.factorial(self.K)
        self.prob_range = numpy.arange(0.01, 1.01, .01)
        self.Threshold = .000000001

    def __call__(self, data):
        """Return log likelihoods for [0.0, 1.0, step=.01]
        """
        vr = []
        for p in self.prob_range:
            vr.append(
                data.apply(
                    prob, 
                    axis=1, 
                    args=(p, self.K, self.K_factorial, self.Threshold),
                ).apply(
                    numpy.log,
                )
            )
        vrmatrix = dask.dataframe.concat(vr, axis=1)
        vrmatrix.columns = self.prob_range
        return vrmatrix
        #vrmatrix = pandas.DataFrame(dict(zip(prob_range, vr)))
        #return numpy.log(vrmatrix)
dask_compute_log_likelihoods = ComputeLogLikelihood()

In [13]:
a = dask_compute_log_likelihoods(dask.dataframe.from_pandas(df, chunksize=1))

/usr/lib/python3/dist-packages/dask/dataframe/core.py:3018: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  warnings.warn(msg)
/usr/lib/python3/dist-packages/dask/dataframe/core.py:2259: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  warnings.warn(msg)


In [14]:
a

,0.01,0.02,0.03,0.04,0.05,0.060000000000000005,0.06999999999999999,0.08,0.09,0.09999999999999999,0.11,0.12,0.13,0.14,0.15000000000000002,0.16,0.17,0.18000000000000002,0.19,0.2,0.21000000000000002,0.22,0.23,0.24000000000000002,0.25,0.26,0.27,0.28,0.29000000000000004,0.3,0.31,0.32,0.33,0.34,0.35000000000000003,0.36000000000000004,0.37,0.38,0.39,0.4,0.41000000000000003,0.42000000000000004,0.43,0.44,0.45,0.46,0.47000000000000003,0.48000000000000004,0.49,0.5,0.51,0.52,0.53,0.54,0.55,0.56,0.5700000000000001,0.5800000000000001,0.59,0.6,0.61,0.62,0.63,0.64,0.65,0.66,0.67,0.68,0.6900000000000001,0.7000000000000001,0.7100000000000001,0.72,0.73,0.74,0.75,0.76,0.77,0.78,0.79,0.8,0.81,0.8200000000000001,0.8300000000000001,0.8400000000000001,0.85,0.86,0.87,0.88,0.89,0.9,0.91,0.92,0.93,0.9400000000000001,0.9500000000000001,0.9600000000000001,0.97,0.98,0.99,1.0
npartitions=91,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
gSpikein_ERCC-00002,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
gSpikein_ERCC-00003,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
gSpikein_ERCC-00170,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
gSpikein_ERCC-00171,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [15]:
def log_likelihood(data, data_runs=None):
    results = []
    if data_runs is None:
        data_runs = data.run.unique()

    likelihoods = compute_log_likelihoods(data)
    print(type(likelihoods))
    for run_name in data_runs:
        results.append(optimize_by_run(data, likelihoods, run_name))

    if len(data.tube_type.unique()) > 1:
        results.append(optimize_by_tube_type(data, likelihoods))

    return pandas.DataFrame(results)


In [16]:
log_likelihood(df, data_runs=q.columns)

<class 'pandas.core.frame.DataFrame'>


,run_name,run_LR,like_non_run,like_run,like_tot,psmc_non_run,psmc_run,psmc_tot,tube_type
0,19908_C1,2.182787e-11,-9054.070004,-288.411908,-9342.481912,1.00,1.00,1.0,single
1,19909_D8,2.182787e-11,-9045.054630,-297.427282,-9342.481912,1.00,1.00,1.0,single
2,19911_F3,2.182787e-11,-9050.034795,-292.447117,-9342.481912,1.00,1.00,1.0,single
3,20032_G3,2.182787e-11,-9048.772903,-293.709009,-9342.481912,1.00,1.00,1.0,single
4,20026_A9,2.182787e-11,-9044.297558,-298.184354,-9342.481912,1.00,1.00,1.0,single
5,20048_E12,2.182787e-11,-9050.351747,-292.130165,-9342.481912,1.00,1.00,1.0,single
6,20046_C10,2.182787e-11,-9047.513731,-294.968181,-9342.481912,1.00,1.00,1.0,single
7,18275_F1,2.182787e-11,-9043.885420,-298.596492,-9342.481912,1.00,1.00,1.0,single
8,18274_E1,2.182787e-11,-9050.463344,-292.018568,-9342.481912,1.00,1.00,1.0,single
9,20038_F3,4.108104e-03,-9049.693878,-292.785980,-9342.481912,0.99,1.00,1.0,single


In [17]:
df = dask.delayed(log_likelihood)(df)
df = df.sort_values(by='run_LR', ascending=True)
df = chi(df)

TypeError: Truth of Delayed objects is not supported

In [ ]:
def foo():    
    return pandas.Series([3,])
d = []
for i in range(10):
    d.append(dask.delayed(foo)())
df2 = dask.dataframe.from_delayed(d)


In [ ]:
q.head()

In [ ]:
def generate_likelihoods(root, experiments, to_exclude):

    for name, replicates in by_cluster.iterrows():
        fpkm_name = name + '_gene_FPKM.csv'
        target_name = name + '_likelihood.tsv'
        fpkm_path = os.path.join(vdir, fpkm_name)
        quantification = pandas.read_csv(fpkm_path, dtype={'gene_id': str, 'gene_name': str})
        quantification.drop('gene_name', axis=1, inplace=True)
        quantification.set_index('gene_id', inplace=True)
        libraries = sorted(set(quantification.columns) - to_exclude)
        filtered = quantification[libraries]
        print(name, fpkm_name, os.path.exists(fpkm_path), quantification.shape[1], filtered.shape[1])


In [ ]:
def load_psmcs():
    order = []
    psmc = []
    for filename in filenames:
        run_name = filenames[filename]
        order.append(run_name)
        data = pandas.read_csv(filename, sep='\t', index_col=0)
        for cell, row in data[['psmc_run']].iterrows():
            psmc.append((run_name, cell, row.psmc_run))

    psmc = pandas.DataFrame(psmc, columns=['label', 'cell_id', 'Psmc'])

    ax = plot_psmc(psmc, order)
    psmc.to_csv('psmc_by_run.tsv', sep='\t', index=False)
    ax.figure.savefig('psmc_by_run.png', bbox_inches="tight", pad_inches=0, transparent=True)

In [ ]:
def plot_psmc(psmc, order=None):
    mm2inch = 25.4
    fontsize = 8
    fontname = 'Arial'
    fontkw = {
        'fontsize': fontsize,
        'fontname': fontname
    }
    f = pyplot.figure(figsize=(65/mm2inch, 85/mm2inch), dpi=150)
    ax = f.add_subplot(1, 1, 1)
    boxplot(
        x='label', y='Psmc', data=psmc,
        color='#c0c0c0',
        linewidth=0.8,
        fliersize=0,
        ax=ax)
    #stripplot(x='label', y='Psmc', color='k', data=psmc,
    #          order=order, jitter=True, s=2)
    ax.legend().remove()
    ax.set_ylim([0, 1])
    ax.set_xlabel('')
    ax.set_ylabel('Psmc', **fontkw)
    for l in ax.xaxis.get_ticklabels():
        l.set_rotation(90)
        l.set_ha('center')
        l.set_fontsize(fontsize)
        l.set_fontname(fontname)

    return ax

In [ ]:
vdir = os.path.expanduser('~/proj/C1_mouse_limb_combined/paper_analysis_vdir')
by_cluster = load_experiments([os.path.join(vdir, 'experiment_run.tsv')])
to_exclude = load_exclude_list()

In [ ]:
filenames = {
    #'C1_e10.5_mouse_limb_run1_June6_2016_FPKM_likelihood.tsv': 'run 1 e10.5',
    #'C1_e10.5_mouse_limb_run2_June20_2016_FPKM_likelihood.tsv': 'run 2 e13.5',
    'C1_e10.5_mouse_limb_run3_Dec5_2016_gene_FPKM_likelihood.tsv': 'run  3 e10.5',
    'C1_mouse_e13.5_limb_mesenchyme_mm10_run4_gene_FPKM_likelihood.tsv': 'run  4 e13.5',
    'C1_mouse_e11.0_limb_mesenchyme_mm10_run5_gene_FPKM_likelihood.tsv': 'run  5 e11.0',
    'C1_mouse_e11.5_limb_mesenchyme_mm10_run6_gene_FPKM_likelihood.tsv': 'run  6 e11.5',
    'C1_mouse_e12.5_limb_mesenchyme_mm10_run7_gene_FPKM_likelihood.tsv': 'run  7 e12.5',
    'C1_mouse_e13.5_limb_mesenchyme_mm10_run8_gene_FPKM_likelihood.tsv': 'run  8 e13.5',
    #'C1_mouse_e12.5_forelimb_run9_March8_2017_rearray_March152018_gene_FPKM_likelihood.tsv': '',
    'C1_mouse_e11.5_forelimb_run10_December11_2017_gene_FPKM_names_likelihood.tsv': 'run 10 e11.5',
    'C1_mouse_e12.0_forelimb_run11_December12_2017_gene_FPKM_names_likelihood.tsv': 'run 11 e12.0',
    'C1_mouse_e13.0_forelimb_run12_December13_2017_gene_FPKM_names_likelihood.tsv': 'run 12 e13.0',
    'C1_mouse_e14.0_forelimb_run13_December14_2017_gene_FPKM_names_likelihood.tsv': 'run 13 e14.0',
    'C1_mouse_e15.5_forelimb_run14_December15_2017_gene_FPKM_names_likelihood.tsv': 'run 14 e15.5',
    'C1_mouse_e10.5_forelimb_run15_January13AM_2018_gene_FPKM_names_likelihood.tsv': 'run 15 e10.5',
    'C1_mouse_e11.0_forelimb_run16_January13PM_2018_gene_FPKM_names_likelihood.tsv': 'run 16 e11.0',
    'C1_mouse_e14.5_forelimb_run17_January16_2018_gene_FPKM_names_likelihood.tsv': 'run 17 e14.5',
}